In [22]:
import os

In [23]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
%cd ..

c:\Users\balub\OneDrive\Desktop


In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [26]:
%pwd 

'c:\\Users\\balub\\OneDrive\\Desktop'

In [27]:
from Vision_Models.constants import *
from Vision_Models.utils.common import read_yaml, create_directories
CONFIG_FILE_PATH


WindowsPath('C:/Users/balub/OneDrive/Desktop/Balaji sample scripts/ML-Deployment-project/config/config.yaml')

In [28]:
class ConfigurationManager:
        def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)

                create_directories([self.config.artifacts_root])

        def get_data_ingestion_config(self) -> DataIngestionConfig:
                config = self.config.data_ingestion

                create_directories([config.root_dir])

                data_ingestion_config = DataIngestionConfig(
                        root_dir=config.root_dir,
                        source_URL=config.source_URL,
                        local_data_file=config.local_data_file,
                        unzip_dir=config.unzip_dir
                )

                return data_ingestion_config


In [29]:
import os
import urllib.request as request
import zipfile
from Vision_Models import logger
from Vision_Models.utils.common import get_size

In [30]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-13 11:37:46,124: INFO: common] yaml file: C:\Users\balub\OneDrive\Desktop\Balaji sample scripts\ML-Deployment-project\config\config.yaml loaded successfully
[2023-11-13 11:37:46,126: INFO: common] yaml file: C:\Users\balub\OneDrive\Desktop\Balaji sample scripts\ML-Deployment-project\params.yaml loaded successfully
[2023-11-13 11:37:46,126: INFO: common] created directory at: artifacts
[2023-11-13 11:37:46,127: INFO: common] created directory at: artifacts/data_ingestion
[2023-11-13 11:37:50,744: INFO: 4037744573] artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A0